In [1]:
# Standard libraries: https://docs.python.org/3/library/
import math
import numpy as np
# Analysis and plotting modules
import pandas as pd
import plotly

# radCAD modules
from radcad import Model, Simulation

import setup

time: 63.1 ms (started: 2022-08-17 12:15:16 +02:00)


In [3]:
from model.stochastic_processes import create_stochastic_process_realizations
import experiments.simulation_configuration as simulation
from experiments.run import run

time: 26.4 ms (started: 2022-08-17 12:16:08 +02:00)


In [4]:
pd.options.plotting.backend = "plotly"

time: 26.2 ms (started: 2022-08-17 12:16:09 +02:00)


## Model

### setup

In [5]:
monte_carlo_runs = 100
timesteps = simulation.TIMESTEPS
dt = simulation.DELTA_TIME

time: 24.3 ms (started: 2022-08-17 12:16:10 +02:00)


In [6]:
initial_price = 2000
strike = initial_price
mu = -0.08
sigma = 0.3
T = 365 * dt  # option maturity / expiration

time: 24.1 ms (started: 2022-08-17 12:16:11 +02:00)


In [7]:
dt

1

time: 34 ms (started: 2022-08-17 12:16:11 +02:00)


In [8]:
volatile_asset_price_samples = create_stochastic_process_realizations(
    "geometric_brownian_motion_process",
    timesteps=timesteps,
    dt=dt,
    mu=mu,
    sigma=sigma,
    initial_price=initial_price,
    runs=monte_carlo_runs,
)

time: 47.8 ms (started: 2022-08-17 12:16:11 +02:00)


In [9]:
initial_state = {
    'risk_free_rate': 0.03,
    'volatile_asset_price': initial_price,
    'discounted_payoff': 0,
}

time: 21.7 ms (started: 2022-08-17 12:16:12 +02:00)


In [10]:
system_params = {
    'dt': [dt],
    'option_maturity': [T],
    'strike_price': [strike],
    #'risk_free_rate': [0.03],
    'volatile_asset_price_process': [lambda run, timestep: volatile_asset_price_samples[run - 1][timestep]]
}

time: 21.5 ms (started: 2022-08-17 12:16:12 +02:00)


## state updates

In [11]:
def update_volatile_asset_price(
    params, substep, state_history, previous_state, policy_input
):
    """Update Volatile Asset Price
    Update the volatile asset price from the `volatile_asset_price_process`.
    """

    # Parameters
    dt = params["dt"]
    volatile_asset_price_process = params["volatile_asset_price_process"]

    # State Variables
    run = previous_state["run"]
    timestep = previous_state["timestep"]

    # Get the price sample for the current run and timestep
    volatile_asset_price_sample = volatile_asset_price_process(run, timestep * dt)

    return "volatile_asset_price", volatile_asset_price_sample

time: 22.6 ms (started: 2022-08-17 12:16:13 +02:00)


In [12]:
def update_discounted_payoff(
    params, substep, state_history, previous_state, policy_input
):
    """Update Volatile Asset Price
    Update the volatile asset price from the `volatile_asset_price_process`.
    """

    # Parameters
    dt = params["dt"]
    volatile_asset_price_process = params["volatile_asset_price_process"]
    option_maturity = params["option_maturity"]
    strike_price = params["strike_price"]

    # State Variables
    run = previous_state["run"]
    timestep = previous_state["timestep"]
    
    volatile_asset_price = previous_state["volatile_asset_price"]
    risk_free_rate = previous_state["risk_free_rate"]

    rf_daily = risk_free_rate / (timesteps * dt)
    d_payoff = max(volatile_asset_price - strike_price, 0) * np.exp(-rf_daily* (T - timestep))

    return "discounted_payoff", d_payoff

time: 23.5 ms (started: 2022-08-17 12:16:13 +02:00)


## PSUBs

In [13]:
state_update_blocks = [
    # Run first
    {
        'policies': {}, # Ignore for now
        # State variables
        'variables': {
            'volatile_asset_price': update_volatile_asset_price,
        }
    },
    {
        'policies': {}, # Ignore for now
        # State variables
        'variables': {
            'discounted_payoff': update_discounted_payoff,
        }
    },
]

time: 23.9 ms (started: 2022-08-17 12:16:15 +02:00)


In [14]:
model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=timesteps, runs=monte_carlo_runs)

time: 25.9 ms (started: 2022-08-17 12:16:21 +02:00)


In [15]:
df, exceptions = run(simulation)

2022-08-17 12:16:23,350 - root - INFO - Running experiment
2022-08-17 12:16:23,551 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-08-17 12:16:23,566 - root - INFO - Starting simulation 0 / run 3 / subset 0
2022-08-17 12:16:23,586 - root - INFO - Starting simulation 0 / run 6 / subset 0
2022-08-17 12:16:23,603 - root - INFO - Starting simulation 0 / run 1 / subset 0
2022-08-17 12:16:23,608 - root - INFO - Starting simulation 0 / run 9 / subset 0
2022-08-17 12:16:23,628 - root - INFO - Starting simulation 0 / run 4 / subset 0
2022-08-17 12:16:23,640 - root - INFO - Starting simulation 0 / run 12 / subset 0
2022-08-17 12:16:23,650 - root - INFO - Starting simulation 0 / run 7 / subset 0
2022-08-17 12:16:23,665 - root - INFO - Starting simulation 0 / run 2 / subset 0
2022-08-17 12:16:23,666 - root - INFO - Starting simulation 0 / run 15 / subset 0
2022-08-17 12:16:23,677 - root - INFO - Starting simulation 0 / run 10 / subset 0
2022-08-17 12:16:23,688 - root - INFO - Startin

## analysis, post processing

In [16]:
df.dropna().query('substep==2').plot(
    x='timestep',
    y=['volatile_asset_price']
)

time: 3.17 s (started: 2022-08-17 12:16:28 +02:00)


In [17]:
df.dropna().query('substep==2').plot(
    x='timestep',
    y=['discounted_payoff']
)

time: 333 ms (started: 2022-08-17 12:16:34 +02:00)


In [18]:
def get_option_payoff(df):
    
    discounted_payoffs = dict()

    for run in df["run"].unique():
        
        df_ = df.query('run == @run')

        d_payoff = df_['discounted_payoff'].iloc[-1]
        
        discounted_payoffs[run] = d_payoff
        
    return pd.DataFrame(discounted_payoffs.items(), columns=['run', 'option_payoff'])

time: 34.3 ms (started: 2022-08-17 12:16:39 +02:00)


In [19]:
payoff_df = get_option_payoff(df)
option_price = payoff_df['option_payoff'].mean()

time: 203 ms (started: 2022-08-17 12:16:40 +02:00)


In [20]:
print('Option price (monte carlo) is:', option_price)

Option price (monte carlo) is: 139.0406825742656
time: 33.7 ms (started: 2022-08-17 12:16:41 +02:00)


# ignore

In [ ]:
class European_Call_Payoff:

    def __init__(self, strike):
        self.strike = strike

    def get_payoff(self, stock_price):
        if stock_price > self.strike:
            return stock_price - self.strike
        else:
            return 0

In [ ]:
ec = European_Call_Payoff(initial_price)

In [ ]:
def get_option_payoff(option, df):
    
    discounted_payoffs = dict()

    for run in df["run"].unique():
        
        df_ = df.query('run == @run')

        va_price = df_['volatile_asset_price'].iloc[-1]
        risk_free_rate = df_['risk_free_rate'].iloc[-1]
        rf_daily = risk_free_rate / (timesteps * dt)
        d_payoff = option.get_payoff(va_price) * np.exp(-rf_daily* T)
        
        discounted_payoffs[run] = d_payoff
        
    return pd.DataFrame(discounted_payoffs.items(), columns=['run', 'option_payoff'])